In [1]:
import requests
from bs4 import BeautifulSoup as bs

def getPageContent(url):
    res = requests.get(url)
    soup = bs(res.text, 'html.parser')
    return soup.select('div.articulum.trans')[0].text.split('googletag')[0].strip()

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import re
import time

localtime = time.strftime("%H/%M")
domain = 'http://www.appledaily.com.tw/'
stopFlag = False

def getPageNews(page=1):
    res = requests.get('http://www.appledaily.com.tw/realtimenews/section/new/{}'.format(page))
    soup = bs(res.text, 'html.parser')

#     print(soup.select('h1.dddd')[0].text)
#     if today==soup.select('h1.dddd')[0].text break

    today = "".join(soup.select('h1.dddd')[0].text.split())
    news = []
    time = today+' 23:59:59'
#     print(len(soup.select('li.rtddt')))
    for li in soup.select('li.rtddt'):
#     print("===================")
        h1 = li.select('h1')[0].text
        m = re.match('(.+)\((\d+)\)', h1)
        if(time < '{} {}'.format(today, li.select('time')[0].text)):
            global stopFlag 
            stopFlag = True
            break
        time  = '{} {}'.format(today, li.select('time')[0].text)
        category = li.select('h2')[0].text
        link = domain + li.select('a')[0]['href']
        if m:
            title =  m.group(1)
            view_cnt = int(m.group(2))
#       print(title, view_cnt, time, category, link)
            news.append({'title':title, 'view_cnt':view_cnt, 'time': time, 'category': category, 'link': link, 'summary': getPageContent(link)}) 
    return news

In [3]:
newsToday=[]
page=1
while True:
    print(page, end=' ')
    newsToday.extend(getPageNews(page));
    if(stopFlag):
        break
    page += 1;    
# print(newsToday)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 

In [4]:
# -*-coding: utf-8 -*-
import sqlite3
db=sqlite3.connect('news.sqlite')
cur=db.cursor()
cur.execute('''DROP TABLE IF EXISTS news_entry''')
cur.execute('''CREATE TABLE IF NOT EXISTS news_entry
     (id INTEGER PRIMARY KEY,
      category TEXT,
      view_cnt INTEGER,
      time TEXT,
      title TEXT,
      summary TEXT,
      link TEXT)''')
cur.close()

In [ ]:
import sqlite3
db=sqlite3.connect('news.sqlite')
cur=db.cursor()
insert_stmt = 'insert into news_entry({}) values({})'
for idx,news in enumerate(newsToday):
    print(idx, end=' ')
    columns = ','.join(news.keys())
    vals = ','.join(len(news.keys()) * '?')
    sql = insert_stmt.format(columns, vals)
#     print(sql)
#     print(news.values())
    cur.execute(sql, list(news.values())) # 要list()轉型
    db.commit() # 要commit
cur.close()
db.close()

# Dataframe to DB

In [12]:
import pandas as pd 
df = pd.DataFrame(newsToday)

import sqlite3 as lite
with lite.connect('news.sqlite') as db:
    df.to_sql(name='news_entry', index=False, con=db, if_exists='replace')
db.close()

In [10]:
# Drop table 
import sqlite3
db=sqlite3.connect('news.sqlite')
cur=db.cursor()
sql = 'drop table news_entry3'
try:
    cur.execute(sql)
except:
    print("Error Occur!")
    traceback.print_exc()
finally:    
    db.commit() # 要commit
    cur.close()
    db.close()